In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import io
import time

# Import Encoder and Decoder

In [0]:
from model import Encoder, LuongAttention, LoungDecoder

# Load Dataset

In [0]:
lines = io.open('/content/gdrive/My Drive/neural machine translation/hin.txt', encoding='UTF-8').read().strip().split('\n')

word_pairs = [[a for a in l.split('\t')]  for l in lines]
for i in range(len(word_pairs)):
  word_pairs[i] = word_pairs[i][:2]

In [5]:
word_pairs[1000:1010]

[['Apples were on sale today.', 'आज सेव सस्ते बिक रहे थे।'],
 ['As for me, I am satisfied.', 'मैं तो संतुष्ट हूँ।'],
 ['Can you come to the party?', 'तुम पार्टी में आ सकते हो क्या?'],
 ['Canada is a large country.', 'कनाडा एक बड़ा देश है ।'],
 ['Come to my house at eight.', 'मेरे घर आठ बजे आना।'],
 ['Did the police arrest Tom?', 'पुलीस ने टॉम को गिरफ़्तार किया क्या?'],
 ['Do you believe in fairies?', 'तुम परियों में विश्वास करते हो क्या?'],
 ['Do you believe in fairies?', 'तुम परियों में विश्वास करती हो क्या?'],
 ['Do you have a larger size?', 'आपके पास इससे बड़े नाप का है क्या?'],
 ['Do you want to come along?', 'तुम साथ आना चाहते हो क्या?']]

# Preprocess Data

In [0]:
def unicode_to_ascii(s):
  return ''.join(c for c in unicodedata.normalize('NFD', s)
    if unicodedata.category(c) != 'Mn')


def preprocess_sentence_en(s):
  s = unicode_to_ascii(s.lower().strip())
  s = re.sub(r"([?.!|,¿])", r" \1 ", s)
  s = re.sub(r'[" "]+', " ", s)
  s = s.strip()
  s = '<start> ' + s + ' <end>'
  return s

def preprocess_sentence_hn(s):
  s = s.lower().strip()
  s = re.sub(r"([?.!|,¿])", r" \1 ", s)
  s = re.sub(r'[" "]+', " ", s)
  s = s.strip()
  s = '<start> ' + s + ' <end>'
  return s  

def tokenize(lang):
  tokenizer = tf.keras.preprocessing.text.Tokenizer(
      filters='')
  tokenizer.fit_on_texts(lang)

  tensor = tokenizer.texts_to_sequences(lang)

  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')

  return tensor, tokenizer

In [0]:
inp_lang = [preprocess_sentence_en(p[0]) for p in word_pairs]
target_lang = [preprocess_sentence_hn(p[1]) for p in word_pairs]

input_tensor, input_lang_tokenizer = tokenize(inp_lang)
target_tensor, target_lang_tokenizer = tokenize(target_lang)

In [8]:
print(input_tensor.shape)
print(target_tensor.shape)

(2869, 27)
(2869, 29)


In [0]:
max_length_targ, max_length_inp = target_tensor.shape[1], input_tensor.shape[1]


In [0]:
BUFFER_SIZE = len(input_tensor)
BATCH_SIZE = 64
steps_per_epoch = len(input_tensor)//BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(input_lang_tokenizer.word_index)+1
vocab_tar_size = len(target_lang_tokenizer.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((input_tensor, target_tensor)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

# Initialize Encoder and Decoder models

In [0]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)
attention_decoder = LoungDecoder(vocab_tar_size, embedding_dim, units, 'concat')


In [24]:
encoder.summary()

Model: "encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  618496    
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
Total params: 5,865,472
Trainable params: 5,865,472
Non-trainable params: 0
_________________________________________________________________


In [25]:
attention_decoder.summary()

Model: "loung_decoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
luong_attention (LuongAttent multiple                  2099201   
_________________________________________________________________
embedding_1 (Embedding)      multiple                  779776    
_________________________________________________________________
lstm_1 (LSTM)                multiple                  5246976   
_________________________________________________________________
dense_2 (Dense)              multiple                  2098176   
_________________________________________________________________
dense_3 (Dense)              multiple                  3122150   
Total params: 13,346,279
Trainable params: 13,346,279
Non-trainable params: 0
_________________________________________________________________


In [0]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

# Training

## Define single train step

In [0]:

def train_step(inp, targ, enc_hidden):
  loss = 0

  with tf.GradientTape() as tape:

    enc_output, enc_hidden_h, enc_hidden_c = encoder(inp, enc_hidden)

    dec_hidden_h = enc_hidden_h
    dec_hidden_c = enc_hidden_c


    for t in range(1, targ.shape[1]):
      # initialize decoder input to previous timestep of target
      dec_input = tf.expand_dims(targ[:, t-1], 1)
      
      predictions, dec_hidden_h, dec_hidden_c, _ = attention_decoder(dec_input, (dec_hidden_h , dec_hidden_c), enc_output)

      loss += loss_function(targ[:, t], predictions)


  batch_loss = (loss / int(targ.shape[1]))

  variables = encoder.trainable_variables + attention_decoder.trainable_variables

  gradients = tape.gradient(loss, variables)

  optimizer.apply_gradients(zip(gradients, variables))

  return batch_loss

In [0]:
def translate(sentence):

  sentence = preprocess_sentence_en(sentence)

  inputs = [input_lang_tokenizer.word_index[i] for i in sentence.split(' ')]
  inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                         maxlen=max_length_inp,
                                                         padding='post')
  inputs = tf.convert_to_tensor(inputs)

  result = ''

  hidden = (tf.zeros([1, units]), tf.zeros([1, units]))

  enc_out, dec_hidden_h, dec_hidden_c = encoder(inputs, hidden)

  dec_input = tf.expand_dims([target_lang_tokenizer.word_index['<start>']], 0)

  for t in range(max_length_targ):
    predictions, dec_hidden_h, dec_hidden_c, _ = attention_decoder(dec_input, (dec_hidden_h, dec_hidden_c), enc_out)



    predicted_id = tf.argmax(predictions[0]).numpy()
    # print(predicted_id)
    result += target_lang_tokenizer.index_word[predicted_id] + ' '

    if target_lang_tokenizer.index_word[predicted_id] == '<end>':
      break

    # the predicted ID is fed back into the model
    dec_input = tf.expand_dims([predicted_id], 0)

  print('Input: %s' % (sentence))
  print('Predicted translation: {}'.format(result))

## Start Training

In [15]:
EPOCHS = 40

for epoch in range(EPOCHS):
  start = time.time()

  enc_hidden = encoder.initialize_hidden_state()
  total_loss = 0

  for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
    batch_loss = train_step(inp, targ, enc_hidden)
    total_loss += batch_loss

    if batch % 100 == 0:
      print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                   batch,
                                                   batch_loss.numpy()))


  print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      total_loss / steps_per_epoch))
  print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 2.1652
Epoch 1 Loss 1.8553
Time taken for 1 epoch 22.59349513053894 sec

Epoch 2 Batch 0 Loss 1.5332
Epoch 2 Loss 1.5627
Time taken for 1 epoch 19.75169086456299 sec

Epoch 3 Batch 0 Loss 1.4556
Epoch 3 Loss 1.4399
Time taken for 1 epoch 20.007569551467896 sec

Epoch 4 Batch 0 Loss 1.3512
Epoch 4 Loss 1.3679
Time taken for 1 epoch 19.680345058441162 sec

Epoch 5 Batch 0 Loss 1.3273
Epoch 5 Loss 1.3101
Time taken for 1 epoch 19.770269870758057 sec

Epoch 6 Batch 0 Loss 1.1678
Epoch 6 Loss 1.2446
Time taken for 1 epoch 19.942848205566406 sec

Epoch 7 Batch 0 Loss 1.1513
Epoch 7 Loss 1.1836
Time taken for 1 epoch 20.038758039474487 sec

Epoch 8 Batch 0 Loss 1.1978
Epoch 8 Loss 1.1303
Time taken for 1 epoch 19.878458738327026 sec

Epoch 9 Batch 0 Loss 1.0825
Epoch 9 Loss 1.0694
Time taken for 1 epoch 20.03354501724243 sec

Epoch 10 Batch 0 Loss 0.9868
Epoch 10 Loss 1.0116
Time taken for 1 epoch 19.868704795837402 sec

Epoch 11 Batch 0 Loss 0.8664
Epoch 11 Loss 0.9522
T

# Translate

In [16]:
translate(u'this is book.')

Input: <start> this is book . <end>
Predicted translation: यह किताब है। <end> 


In [17]:
translate(u'are you at home ?')

Input: <start> are you at home ? <end>
Predicted translation: तुम घर पे हो क्या ? <end> 


In [18]:
translate(u'what is this ?')

Input: <start> what is this ? <end>
Predicted translation: यह क्या है ? <end> 
